In [1]:
%%html
<script>
document.querySelector("#notebook-container div").style.display = 'none';
var timeout = 500; // #1000000ms = 1000 seconds
function ensureKernelIsSet(timeout) {
    var start = Date.now();
    return new Promise(waitFor); 
    function waitFor(resolve, reject) {
        if (Jupyter && Jupyter.notebook && Jupyter.notebook.kernel)
            resolve(Jupyter.notebook.kernel);
        else if (timeout && (Date.now() - start) >= timeout)
            reject(new Error("timeout"));
        else
            setTimeout(waitFor.bind(this, resolve, reject), 30);
    }
}


ensureKernelIsSet(timeout).then(_kernel => {
    Jupyter.notebook.execute_cells([2,4])
});
</script>

### Загрузка библиотек

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib as plt
import scipy
import sklearn
import statsmodels
import pyspark
import pyhive
import subprocess
import os
from dsconnector import get_connect
from fastparquet import ParquetFile
import snappy


### Выбранные датасеты

In [ ]:
datasets = []
my_datasets = get_connect(datasets)
my_datasets

### Примеры каталогов HDFS

Главная директория выводится кодом в начале ячейки ниже, затем необходимо в последующей ячейке написать **"!hdfs dfs -ls *путь однойиз папок из результата выполнения ячейчки ниже*"**, эта операция выведет содержимое этой папки, необходимо будетподставлять полный путь, пока не будут доступны файлы с расширением .snappy.parquet. Полный путь до файла с этим расширением необходимо вставить вместо"*Здесь необходимо вставить путь до файла с расширением:.snappy.parquet*", в закомментированном коде ниже и выполнить ячейку

In [ ]:
# Отображение одного из выбранных каталогов hdfs
exs_hdfs_dir =  subprocess.run(["hdfs", "dfs", "-ls", f"{my_datasets[{ID}]['Соединение']}"], stdout=subprocess.PIPE) 
print(exs_hdfs_dir.stdout.decode('utf-8'))
# '''
# Для чтения файлов типа 'parquet' используйте следующую конструкцию:

# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession, SQLContext
# conf = SparkConf().setAppName('myAppName')
# conf = conf.setAll([('spark.kryoserializer.buffer.max.mb', '1024'),
#                    ('spark.executor.memory', '5gb'),
#                    ("spark.cores.max", "7"),
#                    ("spark.sql.shuffle.partitions","5"),
#                    ("spark.driver.maxResultSize",'5gb'),
#                    ("spark.sql.parquet.compression.codec", "snappy"),
#                    ("spark.rdd.compress", "true")])
# sc = SparkContext(conf=conf)
# sq = SQLContext(sc)
# df = sq.read.parquet("hdfs://*Здесь необходимо вставить путь до файла с расширением:.snappy.parquet*")
# df.registerTempTable("parquet_file")
# df = sq.sql("select * from parquet_file limit 10").collect()
# df = pd.DataFrame(df)
# df

# '''

### Примеры каталогов HIVE

In [ ]:
# Пример отображения данных из hive
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext 
url = f"hive2://{my_datasets[{ID}]['Соединение']}"
sparkSession = (SparkSession
                .builder
                .appName("hive_table")
                .config("hive.metastore.uris", url, conf=SparkConf())
                .config("spark.master", "yarn") 
                .config("spark.executor.memory", "5g") 
                .config("spark.executor.cores", 5) 
                .config("spark.executor.instances", 5) 
                .enableHiveSupport()
                .getOrCreate())
df_load = sparkSession.sql(f"SELECT * FROM {my_datasets[{ID}]['Таблица']} LIMIT 10")
df_load =  df_load.collect()
df =pd.DataFrame(df_load)
df

### Обработка данных